# Goal : Scan the 3D predcitions with the Dpo fold database. 
## However, we will only scan the proteins that did not make it into the first version of the scan 

### A. Make the DB
### B. Run Foldseek
### C. Get the hits with good proba
### D. Report the folds
***

# A

In [ ]:
rsync -avzhe ssh \
conchae@garnatxa.srv.cpd:/home/conchae/PhageDepo_pdb/Dpo_domains.phagedepo.0805.final.tsv \
/media/concha-eloko/Linux/depolymerase_building

In [ ]:
path_pdb = "/media/concha-eloko/Linux/PhageDEPOdetection/PDB_files"
path_CAZY_db = "/media/concha-eloko/Linux/PhageDEPOdetection/Dpo_RefDB_v2"

with open(f"{path_pdb}/folds_CAZY.tsv", "w") as outfile :
    outfile.write(f"Groups\tFolds\tProtein\n")
    for file in os.listdir(path_pdb) :
        if os.path.isdir(f"{path_pdb}/{file}") == True and len(os.listdir(f"{path_pdb}/{file}"))>0:
            fold = folds_dico[file].replace("_" , " ")
            for pdb_file in os.listdir(f"{path_pdb}/{file}") :
                outfile.write(f"{file}\t{fold}\t{pdb_file}\n")


In [ ]:
import os 
path_pdb = "/media/concha-eloko/Linux/PhageDEPOdetection/PDB_files"
path_CAZY_db = "/media/concha-eloko/Linux/PhageDEPOdetection/Dpo_RefDB_v2"

for file in os.listdir(path_pdb) :
    if os.path.isdir(f"{path_pdb}/{file}") == True and len(os.listdir(f"{path_pdb}/{file}"))>0:
        for pdb_file in os.listdir(f"{path_pdb}/{file}") :
            os.system(f"cp {path_pdb}/{file}/{pdb_file} {path_CAZY_db}")

In [ ]:
# Create the database : 

path_pdb_ref = "/media/concha-eloko/Linux/depolymerase_building/pdb_depolymerase_domains"

# Generates the database :

foldseek createdb \
/media/concha-eloko/Linux/PhageDEPOdetection/Dpo_RefDB_v2 \
/media/concha-eloko/Linux/depolymerase_building/Dpo_RefDB_v2/Dpo_RefDB_v2_db

# Generate index files : 

foldseek createindex \
/media/concha-eloko/Linux/depolymerase_building/Dpo_RefDB_v2/Dpo_RefDB_v2_db \ 
/media/concha-eloko/Linux/depolymerase_building/Dpo_RefDB_v2

# Move to server :
rsync -avzhe ssh \
/media/concha-eloko/Linux/depolymerase_building/RefDepo_domains \
conchae@garnatxa.srv.cpd:/home/conchae/depolymerase
    
rsync -avzhe ssh \
/media/concha-eloko/Linux/depolymerase_building/pdb_depolymerase_domains \
conchae@garnatxa.srv.cpd:/home/conchae/depolymerase

***
## B  

> Get a list of the failed Dpos

In [55]:
import pandas as pd
import os 

path_project = "/media/concha-eloko/Linux/depolymerase_building"
dir_out = f"{path_project}/seekfold_phagedepo"

path_info = f"/media/concha-eloko/Linux/depolymerase_building/depolymerase_fold.csv"
info_df = pd.read_csv(path_info , sep = "\t", header = 0)

first_batch_results = set()

outputs = [f"{dir_out}/{file}" for file in os.listdir(dir_out) if file[-3:]=="out"]
header_seekfold = ["query","target","pident","alnlen","gapopen","qstart","qend","tstart","tend","bits","prob"]

for results in outputs :
    results_df = pd.read_csv(f"{results}", sep = "\t" , names = header_seekfold)
    for _,row in results_df.iterrows() :
        fold = info_df[info_df["ID"] == row["target"]]["X_Group_Name"].values[0]
        if fold != "jelly-roll" : #and row["alnlen"] >= 150
            if row["prob"] >= 0.5 :
                first_batch_results.add((row["query"] , fold))
            elif fold == "right-handed beta-helix" and row["prob"] >= 0.2 :
                first_batch_results.add((row["query"] , fold))

first_batch_dpos = {dpo for (dpo,fold) in first_batch_results}



In [58]:
import pandas as pd
import os 

path_project = "/media/concha-eloko/Linux/depolymerase_building"
dir_out = f"{path_project}/seekfold_phagedepo"

path_info = f"/media/concha-eloko/Linux/depolymerase_building/depolymerase_fold.csv"
info_df = pd.read_csv(path_info , sep = "\t", header = 0)

first_batch_results = set()

outputs = [f"{dir_out}/{file}" for file in os.listdir(dir_out) if file[-3:]=="out"]
header_seekfold = ["query","target","pident","alnlen","gapopen","qstart","qend","tstart","tend","bits","prob"]

target_toroid = []
for results in outputs :
    results_df = pd.read_csv(f"{results}", sep = "\t" , names = header_seekfold)
    for _,row in results_df.iterrows() :
        fold = info_df[info_df["ID"] == row["target"]]["X_Group_Name"].values[0]
        if fold != "jelly-roll" : #and row["alnlen"] >= 150
            if row["prob"] >= 0.5 :
                if fold.count("( α / α )") >= 0 :
                    target_toroid.append(row["target"])  
                    break


In [59]:
Counter(target_toroid)

Counter({'AF-A0A658Y395-F1-model_v4.pdb': 357,
         'AF-A0A1E5LEG1-F1-model_v4.pdb': 155,
         'AF-A0A0A6QEK9-F1-model_v4.pdb': 66,
         'AF-A0A0R3Q9P4-F1-model_v4.pdb': 60,
         'AF-A0A654UB90-F1-model_v4.pdb': 56,
         'AF-A0A0B5KP61-F1-model_v4.pdb': 51,
         'AF-A0A7C4BP91-F1-model_v4.pdb': 43,
         'AF-A0A0F6AK51-F1-model_v4.pdb': 39,
         'AF-A0A831UGW4-F1-model_v4.pdb': 38,
         'A1l_00058_A_1_1_128.pdb': 15,
         'AF-A0A496B6X2-F1-model_v4.pdb': 12,
         'AF-A0A562WF65-F1-model_v4.pdb': 7,
         'AF-A0A202B6A7-F1-model_v4.pdb': 5,
         'AF-A0A0E2ALL6-F1-model_v4.pdb': 4,
         'AF-A0A7G2DEW9-F1-model_v4.pdb': 3,
         'AF-A0A0D3A0D6-F1-model_v4.pdb': 2,
         'AF-A0A7K3HGV9-F1-model_v4.pdb': 2,
         'AF-A0A6V8K287-F1-model_v4.pdb': 1,
         'AF-H9BBE9-F1-model_v4.pdb': 1,
         'AF-P49371-F1-model_v4.pdb': 1})

In [ ]:
no length restriction : 957 
150_amino acids : 930 

In [ ]:
import subprocess
import os

path_project = "/media/concha-eloko/Linux/depolymerase_building"
path_pdb = f"{path_project}/phagedepo_out"
path_db = f"/media/concha-eloko/Linux/depolymerase_building/Dpo_RefDB_v2/Dpo_RefDB_v2_db"
path_tmp = f"{path_project}/tmp"

def seek_beta_helix(path_in) :
    dir_out = f"{path_project}/CAZY_seekfold"
    protein_id = path_in.split("/")[-1].split(".pdb")[0]
    path_out = f"{dir_out}/{protein_id}.out"
    output_frmt = f"query,target,pident,alnlen,gapopen,qstart,qend,tstart,tend,bits,prob"
    seek = f"foldseek easy-search {path_in} {path_db} {path_out} {path_tmp} --format-output {output_frmt}"
    #seek = f"foldseek easy-search {path_in} {path_db} {path_out}.html {path_tmp} --format-mode 3"
    seek_process = subprocess.Popen(seek, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
    press_out, press_err = seek_process.communicate()
    print (press_out, press_err)

paths = [f"{path_pdb}/{file}" for file in os.listdir(f"{path_pdb}") if file not in first_batch_dpos]


for path in paths :
    seek_beta_helix(path)

***
## C.

In [ ]:
import pandas as pd
import os 

path_project = "/media/concha-eloko/Linux/depolymerase_building"
dir_out = f"{path_project}/CAZY_seekfold"
outputs = [f"{dir_out}/{file}" for file in os.listdir(dir_out) if file[-3:]=="out"]

header_seekfold = ["query","target","pident","alnlen","gapopen","qstart","qend","tstart","tend","bits","prob"]

depo_ppt = {}
very_good = 0
good = 0
decent = 0 
passable = 0
for results in outputs :
    results_df = pd.read_csv(f"{results}", sep = "\t" , names = header_seekfold)
    try : 
        if results_df["prob"][0] > 0.1 :
            depo_ppt[results_df["query"][0]] = (results_df["target"][0] , results_df["prob"][0])
            if results_df["prob"][0] > 0.9 :
                very_good += 1
            elif results_df["prob"][0] > 0.5 :
                good += 1
            elif results_df["prob"][0] > 0.25 :
                decent += 1
            else :
                passable += 1
                
    except IndexError :
        pass

import pprint
#pp = pprint.PrettyPrinter(width = 100, sort_dicts = True, compact = True)


print(f"So far, out of {len((outputs))} proteins searched, {len(depo_ppt)} seemed to be positive in the phagedepo dataset ({len(depo_ppt)/len(outputs)} %).\nThe proportion of very good is {very_good/len(depo_ppt)}\nThe proportion of good is {good/len(depo_ppt)}\nThe proportion of decent is {decent/len(depo_ppt)}\nThe rest is {passable/len(depo_ppt)}")


In [ ]:
path_info = f"/media/concha-eloko/Linux/PhageDEPOdetection/PDB_files/folds_CAZY.tsv"
info_df = pd.read_csv(path_info , sep = "\t", header = 0)

info_df["Folds"].unique()

***
## C.

In [ ]:
import pandas as pd
import os 

path_project = "/media/concha-eloko/Linux/depolymerase_building"
dir_out = f"{path_project}/CAZY_seekfold"

path_info = f"/media/concha-eloko/Linux/PhageDEPOdetection/PDB_files/folds_CAZY.tsv"
info_df = pd.read_csv(path_info , sep = "\t", header = 0)

dico_folds = {"jelly-roll" : [],
              "alpha/alpha toroid" : [],
              "right-handed beta-helix" : [] ,
              "TIM beta/alpha-barrel" : [],
              "6-bladed beta-propeller" : [] ,
              "Flavodoxin-like" : [] ,
              "Alpha/Beta hydrolase fold" : [] ,
              "Other" : [],
             }

outputs = [f"{dir_out}/{file}" for file in os.listdir(dir_out) if file[-3:]=="out"]
header_seekfold = ["query","target","pident","alnlen","gapopen","qstart","qend","tstart","tend","bits","prob"]
depo_results = {}
for results in outputs :
    results_df = pd.read_csv(f"{results}", sep = "\t" , names = header_seekfold)
    for _,row in results_df.iterrows() :
        if row["prob"] >= 0.5 :
            try : 
                fold = info_df[info_df["Protein"] == row["target"]]["Folds"].values[0]
                if fold != "parallel β-helix" :
                    if row["alnlen"] >= 200 :
                        print(row["target"] , row["query"], fold)
                        break
            except Exception as e :
                pass

In [43]:
import pandas as pd
import os 

path_project = "/media/concha-eloko/Linux/depolymerase_building"
dir_out = f"{path_project}/CAZY_seekfold"

path_info = f"/media/concha-eloko/Linux/PhageDEPOdetection/PDB_files/folds_CAZY.tsv"
info_df = pd.read_csv(path_info , sep = "\t", header = 0)

dico_folds = {"jelly-roll" : [],
              "alpha/alpha toroid" : [],
              "right-handed beta-helix" : [] ,
              "TIM beta/alpha-barrel" : [],
              "6-bladed beta-propeller" : [] ,
              "Flavodoxin-like" : [] ,
              "Alpha/Beta hydrolase fold" : [] ,
              "Other" : [],
             }

outputs = [f"{dir_out}/{file}" for file in os.listdir(dir_out) if file[-3:]=="out"]
header_seekfold = ["query","target","pident","alnlen","gapopen","qstart","qend","tstart","tend","bits","prob"]
second_batch_results = set()
fake_targets = set()

for results in outputs :
    results_df = pd.read_csv(f"{results}", sep = "\t" , names = header_seekfold)
    try : 
        for _,row in results_df.iterrows() :
            if row["target"].count("_") ==0 :
                if row["prob"] >= 0.5 and row["alnlen"] >= 200:
                    if row["target"] not in info_df["Protein"].tolist() :
                        fake_targets.add(row["target"])
                        continue
                    else :
                        #filtered_df = info_df[info_df["Protein"] == row["target"]]
                        fold = info_df[info_df["Protein"] == row["target"]]["Folds"].values[0]
                        if fold != "jelly-roll" :
                            second_batch_results.add((row["query"] , fold))
                            break
                        else :
                            continue
    except Exception as e :
        print(e , row["query"])
        print(results_df)
                
second_batch_dpos = {dpo for (dpo,fold) in second_batch_results}
len(second_batch_dpos)

164

In [44]:
all_hits = first_batch_results.union(second_batch_results)
len(all_hits)

1121

In [54]:
for hit in all_hits : 
    if hit[1] == "( α / α ) 6 barrel" : 
        print(hit)

('721.pdb', '( α / α ) 6 barrel')
('2074.pdb', '( α / α ) 6 barrel')
('3294.pdb', '( α / α ) 6 barrel')
('6511.pdb', '( α / α ) 6 barrel')
('213.pdb', '( α / α ) 6 barrel')
('3348.pdb', '( α / α ) 6 barrel')
('2549.pdb', '( α / α ) 6 barrel')
('3883.pdb', '( α / α ) 6 barrel')
('5536.pdb', '( α / α ) 6 barrel')
('6577.pdb', '( α / α ) 6 barrel')
('2558.pdb', '( α / α ) 6 barrel')
('3803.pdb', '( α / α ) 6 barrel')
('3308.pdb', '( α / α ) 6 barrel')
('5365.pdb', '( α / α ) 6 barrel')
('5367.pdb', '( α / α ) 6 barrel')
('6194.pdb', '( α / α ) 6 barrel')
('3135.pdb', '( α / α ) 6 barrel')
('1672.pdb', '( α / α ) 6 barrel')
('3099.pdb', '( α / α ) 6 barrel')
('3302.pdb', '( α / α ) 6 barrel')
('5363.pdb', '( α / α ) 6 barrel')
('5801.pdb', '( α / α ) 6 barrel')
('3125.pdb', '( α / α ) 6 barrel')
('2575.pdb', '( α / α ) 6 barrel')
('3564.pdb', '( α / α ) 6 barrel')
('5135.pdb', '( α / α ) 6 barrel')
('5351.pdb', '( α / α ) 6 barrel')
('5728.pdb', '( α / α ) 6 barrel')
('6538.pdb', '( α / α 

In [46]:
from collections import Counter 

len(all_hits)
dpos_folds_list = [item[1] for item in all_hits]
Counter(dpos_folds_list)

Counter({'right-handed beta-helix': 828,
         '6-bladed beta-propeller': 118,
         '( β / α ) 8 barrel': 62,
         '( α / α ) 6 barrel': 37,
         '5-fold β-propeller': 17,
         'parallel β-helix': 13,
         'β-propeller': 12,
         '6-bladed β-propeller': 10,
         'β-jelly roll': 9,
         'TIM beta/alpha-barrel': 7,
         'Other': 4,
         '( α / α ) 6 toroid + anti-parallel β-sheet': 2,
         'lysozyme fold': 1,
         '( β / α ) 7 barrel': 1})

In [62]:
import pandas as pd
import os 

path_project = "/media/concha-eloko/Linux/depolymerase_building"
dir_out = f"{path_project}/CAZY_seekfold"

path_info = f"/media/concha-eloko/Linux/PhageDEPOdetection/PDB_files/folds_CAZY.tsv"
info_df = pd.read_csv(path_info , sep = "\t", header = 0)

dico_folds = {"jelly-roll" : [],
              "alpha/alpha toroid" : [],
              "right-handed beta-helix" : [] ,
              "TIM beta/alpha-barrel" : [],
              "6-bladed beta-propeller" : [] ,
              "Flavodoxin-like" : [] ,
              "Alpha/Beta hydrolase fold" : [] ,
              "Other" : [],
             }

info_df

,Groups,Folds,Protein
0,PL6_1,parallel β-helix,5gkd.pdb
1,PL6_1,parallel β-helix,7dmk.pdb
2,PL6_1,parallel β-helix,5gkq.pdb
3,GH16_3,β-jelly roll,2vy0.pdb
4,GH94,( α / α ) 6 barrel,3qde.pdb
...,...,...,...
388,GH43_1,5-fold β-propeller,5gll.pdb
389,GH43_1,5-fold β-propeller,5glk.pdb
390,GH152,β-jelly roll,3zs3.pdb
391,GH152,β-jelly roll,7p20.pdb


In [68]:
import pandas as pd
import os 

path_project = "/media/concha-eloko/Linux/depolymerase_building"
dir_out = f"{path_project}/CAZY_seekfold"

path_info = f"/media/concha-eloko/Linux/PhageDEPOdetection/PDB_files/folds_CAZY.tsv"
info_df = pd.read_csv(path_info , sep = "\t", header = 0)

dico_folds = {"jelly-roll" : [],
              "alpha/alpha toroid" : [],
              "right-handed beta-helix" : [] ,
              "TIM beta/alpha-barrel" : [],
              "6-bladed beta-propeller" : [] ,
              "Flavodoxin-like" : [] ,
              "Alpha/Beta hydrolase fold" : [] ,
              "Other" : [],
             }

outputs = [f"{dir_out}/{file}" for file in os.listdir(dir_out) if file[-3:]=="out"]
header_seekfold = ["query","target","pident","alnlen","gapopen","qstart","qend","tstart","tend","bits","prob"]
target_toroid = []

for results in outputs :
    try : 
        results_df = pd.read_csv(f"{results}", sep = "\t" , names = header_seekfold)
        for _,row in results_df.iterrows() :
            fold = info_df[info_df["Protein"] == row["target"]]["Folds"].values[0]
            if fold != "jelly-roll" and row["alnlen"] >= 150 :
                if row["prob"] >= 0.5 :
                    if fold.count("( α / α )") >= 0 :
                        target_toroid.append(row["target"])  
                        break
    except Exception as e :
        pass 


In [66]:
Counter(target_toroid)

Counter({'4yzp.pdb': 32,
         '1wky.pdb': 23,
         '2y8k.pdb': 11,
         '1ug9.pdb': 7,
         '6u4z.pdb': 6,
         '6k0m.pdb': 3,
         '5awo.pdb': 3,
         '1nkg.pdb': 2,
         '5mqs.pdb': 2,
         '3a0n.pdb': 1,
         '3pe7.pdb': 1,
         '7fgz.pdb': 1,
         '7nde.pdb': 1,
         '3jug.pdb': 1,
         '6kcw.pdb': 1,
         '6num.pdb': 1,
         '5jov.pdb': 1,
         '1vd5.pdb': 1})

In [ ]:
dpos_folds = {item[1] for item in all_hits}
dpos_folds

In [47]:
118+12+17

147

In [ ]:
inclusion = ['( α / α ) 6 barrel',
 '( α / α ) 6 toroid + anti-parallel β-sheet',
 '( α / α ) 7 barrel',
 '( β / α ) 7 barrel',
 '( β / α ) 8 (inferred)',
 '( β / α ) 8 barrel',
 'Other',
 'TIM beta/alpha-barrel',
 'β-sandwich + β-sheet']

hits_originals = {item for item in all_hits if item[1] in inclusion}

In [37]:
for item in hits_originals : 
    if item[1] == "β-sandwich + β-sheet" :
        print(item)

NameError: name 'hits_originals' is not defined

In [ ]:
from Bio import SeqIO

def extract_sequence_from_pdb(pdb_path):
    for record in SeqIO.parse(pdb_path, "pdb-atom"):
        return str(record.seq)

In [ ]:
with open(f"{path_project}/rare_folds.tsv" , "w") as outfile :
    

In [ ]:
path_pdb = "/media/concha-eloko/Linux/depolymerase_building/phagedepo_out"

with open(f"{path_project}/rare_folds.tsv" , "w") as outfile :    
    outfile.write(f"Fold\tprotein_id\taa_sequence\n")
    for dpo_item in hits_originals :
        try : 
            seq = extract_sequence_from_pdb(f"{path_pdb}/{dpo_item[0]}")
            os.system(f"cp {path_pdb}/{dpo_item[0]} /media/concha-eloko/Linux/depolymerase_building/rare_folds_pdb")
            outfile.write(f"{dpo_item[1]}\t{dpo_item[0]}\t{seq}\n")
        except Exception as e :
            pass

In [ ]:
pp = pprint.PrettyPrinter(width = 100, sort_dicts = True, compact = True)

pp.pprint(decipher_phagedepo)

***
# Make the CAZY DB